# Characterizing Distributions of Generated Molecules

## Configuration

In [ ]:
import subprocess
import os
print('Current conda environment:', os.environ['CONDA_DEFAULT_ENV'])
os.environ['TOKENIZERS_PARALLELISM'] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # set gpu

cwd = os.getcwd()
print('Working directory:', cwd)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)

In [ ]:
import numpy as np
from numpy.linalg import norm

import pandas as pd

import scipy.stats as st

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, PandasTools
from rdkit.Chem.Draw import MolsToGridImage, MolsMatrixToGridImage
from rdkit import DataStructs
import useful_rdkit_utils as uru

import pickle

from molscore import MolScore
from moleval.metrics.metrics import GetMetrics

from fcd import get_fcd

import medchem as mc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold

In [ ]:
#Seaborn settings for visualizations

rc = {
    "axes.facecolor": "#f7f9fc",
    "figure.facecolor": "#f7f9fc",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

default_palette = 'tab10'

sns.set(rc=rc)
pd.set_option('display.max_columns', 35)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Set pandas display options to improve readability

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

In [ ]:
fragments = []

f = open("data/fragments.smi", "r")

for i in range(1, 51):
    mol = f.readline()
    fragments.append(mol[:-1])

In [ ]:
models = ['reinvent', 'crem', 'coati', 'safe']

## Generating Data

In [ ]:
# %%capture

# reinvent_distributions, crem_distributions, coati_distributions, safe_distributions = [], [], [], []

# for fragment in fragments:
#     for model in models:

#         DF_FILEPATH = f'data/{model}_dataframe.csv'

#         arg1 = '--model'
#         arg2 = '--input_frag'
#         arg3 = '--sample'

#         args = ['python3', 'generate_analogs.py',
#                 arg1, model,
#                 arg2, fragment,
#                 arg3, '200']

#         # Change directory to generate analogs with python script
#         %cd ..

#         subprocess.run(args,
#                     stdout=subprocess.DEVNULL,
#                     stderr=subprocess.STDOUT)
                
#         # Change directory back to that of the current notebook
#         %cd experiments

#         df = pd.read_csv(DF_FILEPATH, index_col=0)

#         df['Model'] = model

#         if model == 'reinvent':
#             reinvent_distributions.append(df)
#         elif model == 'crem':
#             crem_distributions.append(df)
#         elif model == 'coati':
#             coati_distributions.append(df)
#         elif model == 'safe':
#             safe_distributions.append(df)

In [ ]:
# data = {'reinvent' : reinvent_distributions,
#         'crem' : crem_distributions,
#         'coati' : coati_distributions,
#         'safe' : safe_distributions}

In [ ]:
# with open('lists.pkl', 'wb') as file:
#     pickle.dump(data, file)

In [ ]:
with open('data/lists.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
reinvent_distributions = data['reinvent']
crem_distributions = data['crem']
coati_distributions = data['coati']
safe_distributions = data['safe']

In [ ]:
for d in data:

    for i, df in enumerate(data[d]):

        df['Input Fragment'] = fragments[i]

## Defining Utils

In [ ]:
def sparse_int_vect_to_numpy(sparse_vect, length):
    array = np.zeros(length, dtype=int)
    for idx, value in sparse_vect.GetNonzeroElements().items():
        array[idx] = value
    return array

In [ ]:
def remove_odd_rings(df):

    if df.empty: return df

    ring_system_lookup = uru.RingSystemLookup.default()
    df['ring_systems'] = df.SMILES.apply(ring_system_lookup.process_smiles)
    df[['min_ring','min_freq']] = df.ring_systems.apply(uru.get_min_ring_frequency).to_list()

    odd_rings = df['min_freq'] < 100

    df['Odd rings'] = ~ odd_rings

    df_good_rings = df[df['Odd rings']].drop(columns=['Odd rings'])
    
    return df_good_rings.loc[:, ['SMILES', 'Model', 'Input Fragment']]

In [ ]:
def remove_broken_molecles(df):

    smiles = df['SMILES'].to_list()

    mols = [Chem.MolFromSmiles(smile) for smile in smiles]

    df['ROMol'] = mols

    df.dropna(subset=['ROMol'], inplace=True)

    df['num_frags'] = df.ROMol.apply(uru.count_fragments)

    df_single_frag = df.query("num_frags == 1").copy()

    return df_single_frag

In [ ]:
def compute_fcd(fragment, smiles): 
    frag_mol = Chem.MolFromSmiles(fragment)
    canon_frag = Chem.MolToSmiles(frag_mol)

    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    canon_smiles = [Chem.MolToSmiles(mol) for mol in mols]

    fcd = get_fcd(canon_frag, canon_smiles)

    return fcd

In [ ]:
def tanimoto_similarity(smi_1, smi_2):
    fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048,countSimulation=True)
    mol_1 = Chem.MolFromSmiles(smi_1)
    mol_2 = Chem.MolFromSmiles(smi_2)
    
    fp_1 = rdFingerprintGenerator.GetCountFPs([mol_1])[0]
    fp_2 = rdFingerprintGenerator.GetCountFPs([mol_2])[0]
        
    return DataStructs.TanimotoSimilarity(fp_1, fp_2)

In [ ]:
def cosine_similarity(smi_1, smi_2):
    fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048,countSimulation=True)
    mol_1 = Chem.MolFromSmiles(smi_1)
    mol_2 = Chem.MolFromSmiles(smi_2)
    
    fp1 = rdFingerprintGenerator.GetCountFPs([mol_1])[0]
    fp2 = rdFingerprintGenerator.GetCountFPs([mol_2])[0]

    A = sparse_int_vect_to_numpy(fp1, 2048)
    B = sparse_int_vect_to_numpy(fp2, 2048)

    return np.dot(A, B) / (norm(A) * norm(B))

In [ ]:
def compute_similarities(fragment, smiles):

    size = len(smiles)
    snn = 0
    avg_tanimoto = 0
    avg_cosine = 0

    for smile in smiles:

        tanimoto = tanimoto_similarity(fragment, smile)
        cosine = cosine_similarity(fragment, smile)

        avg_tanimoto += tanimoto / size
        avg_cosine += cosine / size

        if tanimoto > snn: snn = tanimoto

    return snn, avg_tanimoto, avg_cosine

In [ ]:
def process_filters(df):

    size = len(df)

    smiles = df['SMILES'].to_list()
    mols = [Chem.MolFromSmiles(smile) for smile in smiles]

    lipinski = [mc.rules.basic_rules.rule_of_five(mol) for mol in mols]

    # Apply some default medchem filters
    dundee = mc.functional.alert_filter(
        mols=mols,
        alerts=["Dundee"],
        n_jobs=-1,
        progress=True,
        return_idx=False,
    )
    
    lily = mc.functional.lilly_demerit_filter(
        mols=mols,
        n_jobs=-1,
        progress=True,
        return_idx=False,
    )

    lipinski_score = np.sum(lipinski) / size
    dundee_score = np.sum(dundee) / size
    lily_score = np.sum(lily) / size

    return lipinski_score, dundee_score, lily_score

In [ ]:
'''
Removes distributions with zero molecules

Removes invalid SMILES strings

Removes molecules with multiple fragments

Removes odd ring systems from distributions
'''
def preprocess_data(data):

    for d in data:
        
        for i, df in enumerate(data[d]):
            
            if len(df) < 30:
                pass
            else:
                smiles_list = df['SMILES'].to_list()

                valid_smiles = []
                invalid_smiles = []

                for smiles in smiles_list:
                        
                    try:
                        molecule = Chem.MolFromSmiles(smiles, sanitize=True)
                        if molecule is not None:
                            valid_smiles.append(True)
                        else:
                            valid_smiles.append(False)
                    except Exception as e:
                        invalid_smiles.append(smiles)

                df_1 = remove_broken_molecles(df)

                df_2 = remove_odd_rings(df_1)

                data[d][i] = df_2
            
        filtered_df_list = [df for df in data[d] if len(df) > 30]
    
        data[d] = filtered_df_list
                
    
    return data

In [ ]:
def compute_metrics(data):

    metrics_df = pd.DataFrame()

    for d in data:
        
        for df in data[d]:
            
            size = len(df)

            fragment = df['Input Fragment'].to_list()[0]
            smiles = df['SMILES'].to_list()

            MetricEngine = GetMetrics(
                n_jobs=1,
                device='cpu',
                batch_size=512,
                run_fcd=False,
                train=fragment
            )
            metrics = MetricEngine.calculate(
                smiles,
                calc_valid=True,
                calc_unique=True,
                unique_k=10000,
                se_k=1000,
                sp_k=1000,
                properties=True,
            )

            # Compute FreChet Distance

            fcd = compute_fcd(fragment, smiles)

            # Compute Single Nearest Neighbor Similarity

            snn, avg_tanimoto, avg_cosine = compute_similarities(fragment, smiles)
            lipinski_score, dundee_score, lily_score = process_filters(df)


            metrics['FCD'] = fcd
            metrics['SNN'] = snn
            metrics['Avg Tanimoto'] = avg_tanimoto
            metrics['Avg Cosine'] = avg_cosine
            metrics['Lipinski Score'] = lipinski_score
            metrics['Dundee Score'] = dundee_score
            metrics['Lily Score'] = lily_score
            metrics['Model'] = d

            temp_df = pd.DataFrame(data=metrics, index=[0])

            metrics_df = pd.concat((metrics_df, temp_df))

    return metrics_df

In [ ]:
def construct_confidence_intervals(dataframes, models):

    stats = {}

    for col in cols:
        stats[col] = []

    for df in dataframes:
        for col in cols:

            samples = df[col].to_numpy()
            mean = round(np.mean(samples),3)

            confidence_interval = st.norm.interval(confidence=0.95,
                                                loc=mean,
                                                scale=st.sem(samples))
            
            int_length = round(.5 * (confidence_interval[1] - confidence_interval[0]), 3)

            stats[col].append((mean, int_length))

    stats_df = pd.DataFrame(data=stats)

    stats_df['Model'] = models

    return stats_df

In [ ]:
def make_predictions(metrics_df, cols):

    X = metrics_df.loc[:, cols]
    y = metrics_df['Model'].to_numpy().reshape(-1, 1)

    enc = OneHotEncoder(handle_unknown='ignore')

    enc.fit_transform(y)

    rf = RandomForestClassifier(n_estimators=100,
                                class_weight='balanced',
                                random_state=1)
    
    steps = [
        ('scaler', StandardScaler()),  # Data preprocessing step
        ('classifier', rf)  # Model step
    ]

    pipeline = Pipeline(steps)

    scoring = ['precision_macro', 'recall_macro', 'f1_macro']
    cv = StratifiedKFold(n_splits=5)
    scores = cross_validate(rf, X, y, cv=cv, scoring=scoring)

    avg_precision, precision_std = scores['test_precision_macro'].mean(), scores['test_precision_macro'].std()
    avg_recall, recall_std = scores['test_recall_macro'].mean(), scores['test_recall_macro'].std()
    avg_f1, f1_std = scores['test_f1_macro'].mean(), scores['test_f1_macro'].std()

    return [(avg_precision, precision_std), (avg_recall, recall_std), (avg_f1, f1_std)]

In [ ]:
def plot_feature_importance(features, feature_importance):

    d = {'Features': features, 'Weights': feature_importance}

    rf_features = pd.DataFrame(data=d).sort_values(['Weights'], ascending=False, ignore_index=True)

    fig, ax = plt.subplots()

    sns.barplot(x = 'Features',
                y = 'Weights',
                data = rf_features,
                palette=default_palette,
                ax=ax)

    # plt.title('Ranking of Random Forest Features')
    ax.set_xlabel('Features', fontsize=20)
    ax.set_ylabel('Feature Importance', labelpad=25, fontsize=20)

    plt.xticks(rotation=90)

    return ax

## Computing Distribution Metrics

In [ ]:
data = preprocess_data(data)

reinvent_distributions = data['reinvent']
crem_distributions = data['crem']
coati_distributions = data['coati']
safe_distributions = data['safe']

In [ ]:
# data['safe'].pop(8)
# data['safe'].pop(7);

In [ ]:
# %%capture

# metrics_df = compute_metrics(data)

# metrics_df.to_csv('data/metrics_df')

In [ ]:
metrics_df = pd.read_csv('data/metrics_df', index_col=0)

metrics_df.reset_index(drop=True, inplace=True)

## Exploratory Data Analysis

### Preliminary Statistics

In [ ]:
metrics_df.head()

In [ ]:
metrics_df['Avg_Filter_Score'] = (metrics_df['Lipinski Score'] + metrics_df['Dundee Score'] + metrics_df['Lily Score']) / 3

In [ ]:
cols = ['IntDiv1', 'IntDiv2', 'SEDiv', 'SPDiv', 'ScaffDiv', 'FG', 'RS', 'Purchasable_ZINC20', 'FCD', 'SNN', 'Avg Tanimoto', 'Avg Cosine', 'Filters', 'Lipinski Score', 'Dundee Score', 'Lily Score', 'Avg_Filter_Score']

In [ ]:
metrics_df[['Model'] + cols].groupby("Model").mean()


In [ ]:
metrics_df[['Model'] + cols].groupby("Model").std()

#### Observations:

 * Uniqueness: Interestingly, COATI and CReM generate distributions which have a uniqueness score of 1, whereas REINVENT and SAFE seem to be generating more duplicates (with SAFE generating the most). It is also interesting to note that the standard deviation of uniqueness for SAFE distributions is much higher than the rest, indicating a high sensitivity for this metric to the input fragment.

 * Diversity: CReM tends to generate the most diverse distributions, whereas SAFE seems to generate the least diverse distributions. All models seem to have fairly consistent diversity metrics; however, it is interesting to note the high standard deviation in sphere exclusion diversity score for CReM.

 * Purchasability: Interestingly, CReM and REINVENT tend to generate purchasable molecules at a higher rate than the rest of the models, while SAFE has an average purchasability rate of 0 across all distributions.

### Calculating Confidence Intervals

In [ ]:
reinvent_df = metrics_df.loc[metrics_df['Model'] == 'reinvent']
crem_df = metrics_df.loc[metrics_df['Model'] == 'crem']
coati_df = metrics_df.loc[metrics_df['Model'] == 'coati']
safe_df = metrics_df.loc[metrics_df['Model'] == 'safe']

dataframes = [reinvent_df, crem_df, coati_df, safe_df]

In [ ]:
confidence_intervals = construct_confidence_intervals(dataframes, models)

### Missing Values

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(metrics_df.isna(), aspect="auto", interpolation="nearest", cmap="gray")
plt.xlabel("Column Number")
plt.ylabel("Sample Number");

It seems like there are no missing values in the dataset.

### Visualizing the distributions (of distributions)

In [ ]:
fig, axs = plt.subplots(nrows=len(cols),
                        ncols=3,
                        figsize=(26, len(cols)*4.2),
                        gridspec_kw={'width_ratios' : [0.4, .3, .3]})

for i, col in enumerate(cols):

    ax = axs[i, 0]

    sns.kdeplot(data=metrics_df[[col,'Model']],
                x=col,
                hue='Model',
                warn_singular=False,
                ax=ax)
    
    ax.set_title(f"\n{col}", fontsize = 9)
    ax.grid(visible=True, which = 'both', linestyle = '--', color='lightgrey', linewidth = 0.75)
    ax.set(xlabel = '', ylabel = '')
    
    ax = axs[i, 1]

    sns.boxplot(data=metrics_df,
                x = 'Model',
                y=col,
                ax=ax,
                width=0.25,
                linewidth=0.90,
                fliersize=2.25)
    
    ax.set(xlabel = '', ylabel = '')
    ax.set_title(f"{col}: Boxplot Grouped by Model", fontsize = 9)

    ax = axs[i, 2]

    intervals = confidence_intervals[col]

    for j, d in enumerate(intervals, start=1):

        mean = d[0]
        confidence_interval = (mean - d[1], mean + d[1])

        # Plot the confidence interval as a line
        ax.plot([j, j], confidence_interval, lw=1.5, color='blue')

        # Plot the mean as a dot
        ax.plot(j, mean, 'x', color='red')

    # Customize the plot
    ax.set_xlim(0.5, 4.5)
    ax.set_xticks([1, 2, 3, 4])
    ax.set_xticklabels(['reinvent', 'crem', 'coati', 'safe'])
    # axs[i, 2].sharey(axs[i, 1])
    ax.set_title(f"{col}: Mean and 95% Confidence Interval by Model", fontsize = 9)

# Adjust layout
plt.tight_layout()

It seems like the models can be consistently differentiated. Some observations:

* Sometimes SAFE produces distributions of molecules which do not pass filters. Why?
* Confidence intervals are especially wide for certain models/metrics:
    * CReM: Sphere Exclusion Diversity, Average Tanimoto Similarity, Average Cosine Similarity
    * SAFE: Filtering scores
* Confidence interval for REINVENT is especially narrow when considering functional group diversity 

### Analyzing Filtering Scores

In [ ]:
filters = ['Lipinski Score', 'Dundee Score', 'Lily Score']
dataframes = [reinvent_df, crem_df, coati_df, safe_df]

scores = {'reinvent' : [],
          'crem' : [],
          'coati' : [],
          'safe' : []}

for df, model in zip(dataframes, scores):

    size = df['#'].sum()

    for filter in filters:

        filter_score = (df[filter] * df['#']).sum() / size

        scores[model].append(filter_score)

In [ ]:
#make bar plots of the scores color coded by model for each filter

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

for i, filter in enumerate(['Lipinski', 'Dundee', 'Lily']):
        
    sns.barplot(x=models, y=[scores[model][i] for model in models], ax=ax[i], palette=default_palette)
        
    ax[i].set_title(filter)
    ax[i].set_ylabel('Fraction of Molecules Passing Filter')

plt.tight_layout()
plt.show()

In [ ]:
bad_dfs = metrics_df.sort_values(['Avg_Filter_Score'])[:3]
good_dfs = safe_df.sort_values(['Avg_Filter_Score'], ascending=False)[:3]

In [ ]:
indices = (bad_dfs.index - 148).to_list()
bad_dfs = [safe_distributions[i] for i in indices]

bad_input_mols = []
bad_mols = []
all_mols = []

for df in bad_dfs:

    input_frag = df['Input Fragment'].to_list()[0]
    input_mol = Chem.MolFromSmiles(input_frag)

    three_smiles = df['SMILES'].sample(3).to_list()
    three_mols = [Chem.MolFromSmiles(smile)for smile in three_smiles]

    mols = [input_mol] + three_mols
    all_mols.append(mols)

MolsMatrixToGridImage(all_mols, subImgSize=(300, 300))

In [ ]:
indices = (good_dfs.index - 148).to_list()
good_dfs = [safe_distributions[i] for i in indices]

good_input_mols = []
good_mols = []
all_mols = []

for df in good_dfs:

    input_frag = df['Input Fragment'].to_list()[0]
    input_mol = Chem.MolFromSmiles(input_frag)

    three_smiles = df['SMILES'].sample(3).to_list()
    three_mols = [Chem.MolFromSmiles(smile)for smile in three_smiles]

    mols = [input_mol] + three_mols
    all_mols.append(mols)

MolsMatrixToGridImage(all_mols, subImgSize=(300, 300))

### Assessing correlation

In [ ]:
y = metrics_df['Model'].to_numpy().reshape(-1, 1)

In [ ]:
enc = OrdinalEncoder()

ord = enc.fit_transform(y)

In [ ]:
X = metrics_df[cols]

X['Model'] = ord.squeeze()

In [ ]:
# Computes feature correlation
corr = X.corr(method="pearson")

In [ ]:
# Create labels for the correlation matrix
labels = np.where(np.abs(corr)>0.75, "S",
                  np.where(np.abs(corr)>0.5, "M",
                           np.where(np.abs(corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(corr, mask=np.eye(len(corr)), square=True,
            center=0, annot=True, fmt='.2f', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

It's somewhat unsurprising that a lot of the variables are highly correlated, considering the fact that many of them measure similar quantities. Some nontrivial relationships:

 * High correlation between purchasability and internal diversity
 * High correlation between functional group and ring system diversity

## Random Forest Classification

Training a random forest to classify which model generated a given distribution based on the metrics calculated above. Outlined is the methodology:

#### Initial Classification

* Normalize FCD column using min-max scaler
* Train random forest with 100 base estimators and balanced class weights; validate using 5-fold stratified cross-validation
* Compute average precision, recall, and f1-score as well as standard deviations for each across 5 folds
* Fit random forest on entire dataset to determine relative feature importances

#### Classification w/ Feature Selection and Correlation Analysis

### Initial Classification

In [ ]:
[(avg_precision, precision_std),
 (avg_recall, recall_std),
 (avg_f1, f1_std)] = make_predictions(metrics_df, cols)

In [ ]:
print(f"Average Precision: {avg_precision:.3f}, Standard Deviation: {precision_std:.3f}")
print(f"Average Recall: {avg_recall:.3f}, Standard Deviation: {recall_std:.3f}")
print(f"Average F1: {avg_f1:.3f}, Standard Deviation: {f1_std:.3f}")

In [ ]:
X = metrics_df.loc[:, cols]
y = metrics_df['Model'].to_numpy().reshape(-1, 1)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit_transform(y);

In [ ]:
rf = RandomForestClassifier(n_estimators=100,
                            class_weight='balanced',
                            random_state=1)

In [ ]:
rf.fit(X, y)

In [ ]:
plot_feature_importance(cols, rf.feature_importances_)

### Classification w/ Feature Selection and Correlation Analysis

In [ ]:
cols2 = ['IntDiv1', 'FG', 'RS', 'FCD', 'SNN', 'Lipinski Score', 'Dundee Score', 'Lily Score']

In [ ]:
[(avg_precision, precision_std),
 (avg_recall, recall_std),
 (avg_f1, f1_std)] = make_predictions(metrics_df, cols2)

print(f"Average Precision: {avg_precision:.3f}, Standard Deviation: {precision_std:.3f}")
print(f"Average Recall: {avg_recall:.3f}, Standard Deviation: {recall_std:.3f}")
print(f"Average F1: {avg_f1:.3f}, Standard Deviation: {f1_std:.3f}")

In [ ]:
X = metrics_df.loc[:, cols2]
y = metrics_df['Model'].to_numpy().reshape(-1, 1)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit_transform(y);

In [ ]:
rf = RandomForestClassifier(n_estimators=100,
                            class_weight='balanced',
                            random_state=1)

In [ ]:
rf.fit(X, y)

In [ ]:
plot_feature_importance(cols2, rf.feature_importances_)

## Making Plots

In [ ]:
cols

In [ ]:
fig, axs = plt.subplots(nrows=3,
                        ncols=1,
                        figsize=(12, 3*4.2))

col = 'Avg_Filter_Score'

ax = axs[0]

sns.kdeplot(data=metrics_df[[col,'Model']],
            x=col,
            hue='Model',
            warn_singular=False,
            ax=ax)
    
ax.set_title(f"\n{col}", fontsize = 9)
ax.grid(visible=True, which = 'both', linestyle = '--', color='lightgrey', linewidth = 0.75)
ax.set(xlabel = '', ylabel = '')
    
ax = axs[1]

sns.boxplot(data=metrics_df,
            x = 'Model',
            y=col,
            ax=ax,
            width=0.25,
            linewidth=0.90,
            fliersize=2.25)
    
ax.set(xlabel = '', ylabel = '')
ax.set_title(f"{col}: Boxplot Grouped by Model", fontsize = 9)

ax = axs[2]

intervals = confidence_intervals[col]

for j, d in enumerate(intervals, start=1):

    mean = d[0]
    confidence_interval = (mean - d[1], mean + d[1])

    # Plot the confidence interval as a line
    ax.plot([j, j], confidence_interval, lw=1.5, color='blue')

    # Plot the mean as a dot
    ax.plot(j, mean, 'x', color='red')

# Customize the plot
ax.set_xlim(0.5, 4.5)
ax.set_xticks([1, 2, 3, 4])
ax.set_xticklabels(['reinvent', 'crem', 'coati', 'safe'])
# axs[i, 2].sharey(axs[i, 1])
ax.set_title(f"{col}: Mean and 95% Confidence Interval by Model", fontsize = 9)
    
    
# Adjust layout
plt.tight_layout()

In [ ]:
metrics = ['IntDiv1', 'FG', 'Avg_Filter_Score']

In [ ]:
set1_df1 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'reinvent']['IntDiv1']})
set1_df2 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'crem']['IntDiv1']})
set1_df3 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'coati']['IntDiv1']})
set1_df4 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'safe']['IntDiv1']})

set2_df1 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'reinvent']['FG']})
set2_df2 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'crem']['FG']})
set2_df3 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'coati']['FG']})
set2_df4 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'safe']['FG']})

set3_df1 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'reinvent']['Avg_Filter_Score']})
set3_df2 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'crem']['Avg_Filter_Score']})
set3_df3 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'coati']['Avg_Filter_Score']})
set3_df4 = pd.DataFrame({'Score': metrics_df[metrics_df['Model'] == 'safe']['Avg_Filter_Score']})

# Combine dataframes into a single dataframe with labels
df1 = pd.concat([set1_df1.assign(Metric='Internal Diversity', Model='reinvent'),
                 set1_df2.assign(Metric='Internal Diversity', Model='crem'),
                 set1_df3.assign(Metric='Internal Diversity', Model='coati'),
                 set1_df4.assign(Metric='Internal Diversity', Model='safe')],
                ignore_index=True)

df2 = pd.concat([set2_df1.assign(Metric='Functional Group Diversity', Model='reinvent'),
                 set2_df2.assign(Metric='Functional Group Diversity', Model='crem'),
                 set2_df3.assign(Metric='Functional Group Diversity', Model='coati'),
                 set2_df4.assign(Metric='Functional Group Diversity', Model='safe')],
                ignore_index=True)

df3 = pd.concat([set3_df1.assign(Metric='Average Filter Score', Model='reinvent'),
                 set3_df2.assign(Metric='Average Filter Score', Model='crem'),
                 set3_df3.assign(Metric='Average Filter Score', Model='coati'),
                 set3_df4.assign(Metric='Average Filter Score', Model='safe')],
                ignore_index=True)

# Combine all into one dataframe
all_data = pd.concat([df1, df2, df3])

# Create the boxplot
plt.figure(figsize=(10, 6))
sns.boxenplot(x='Metric', y='Score', hue='Model', data=all_data, palette=default_palette, k_depth='full')
# plt.title('Grouped Boxplot of Value Distributions by Set')
plt.legend(loc='upper left', bbox_to_anchor=(1, .6))
plt.show()

In [ ]:
metrics_df['FCD'] = metrics_df['FCD'] / metrics_df['FCD'].max()
cols.remove('Avg_Filter_Score')
cols.remove('Filters')

In [ ]:
std_df = metrics_df[['Model'] + cols].groupby("Model").std()

In [ ]:
std_df

In [ ]:
var_reinvent = (std_df.loc['reinvent']**2).mean()
var_crem = (std_df.loc['crem']**2).mean()
var_coati = (std_df.loc['coati']**2).mean()
var_safe = (std_df.loc['safe']**2).mean()

In [ ]:
#make bar plots of the scores color coded by model for each filter

fig, ax = plt.subplots(1, 1, figsize=(5, 5))


sns.barplot(x=models, y=[var_reinvent, var_crem, var_coati, var_safe], ax=ax, palette=default_palette)
        
ax.set_ylabel('Average Variance across Metrics')

plt.tight_layout()
plt.show()